In [52]:
%pip install python-dotenv 'pydantic-ai[logfire]'
from IPython.display import clear_output ; clear_output()

In [53]:
from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

import logfire
_ = logfire.configure(console=False)
_ = logfire.instrument_openai()

In [54]:
from enum import Enum
from textwrap import dedent
from typing import List

from pydantic import BaseModel, Field
from pydantic_ai import Agent, CallContext

In [56]:
class Question(BaseModel):
    reflection: str = Field(..., description='Considering the questions and answers so far, what are things we can ask next?')
    question: str = Field(..., description='The question to ask the other player')

asking_agent = Agent('openai:gpt-4o', result_type=Question)

@asking_agent.system_prompt  
async def asking_agent_system_prompt(ctx: CallContext[List]) -> str:
    turns = ctx.deps
    prompt = dedent(f"""
        You are playing a game of 20 questions.
        You are trying to guess the object the other player is thinking of.
        In each turn, you can ask a yes or no question.
        The other player will answer with "yes", "no".
    """).strip()
    if len(turns) > 0:
        prompt += f"\nHere are the questions you have asked so far and the answers you have received:\n"
        prompt += '\n'.join([' * ' + turn for turn in turns])
    return prompt

class Answer(str, Enum):
    YES = 'yes'
    NO = 'no'
    YOU_WIN = 'you win'

class AnswerResponse(BaseModel):
    reflection: str = Field(..., description=(
        'Considering the question, what is the answer? '
        'Is it "yes" or "no"? Or did they guess the '
        'object and the answer is "you win"?'))
    answer: Answer = Field(..., description='The answer to the question - "yes", "no", or "you win"')

ansering_agent = Agent('openai:gpt-4o', result_type=AnswerResponse)

@ansering_agent.system_prompt
async def answering_agent_system_prompt(ctx: CallContext[str]) -> str:
    prompt = dedent(f"""
        You are playing a game of 20 questions.
        The other player is trying to guess the object you are thinking of.
        The object you are thinking of is: {ctx.deps}.
        Answer with "yes" or "no", or "you win" if the other player has guessed the object.
    """).strip()
    return prompt

def twenty_questions(mytery_object):
    turns = []
    while True:
        question = asking_agent.run_sync('Ask the next question', deps=turns).data.question
        answer = ansering_agent.run_sync(question, deps=mytery_object).data.answer.value
        if answer == Answer.YOU_WIN:
            print('You Win!')
            break
        elif len(turns) >= 20:
            print('You Lose!')
            break
        else:
            turns.append(f'{question} - {answer}')
            print(f'{len(turns)}. QUESTION: {question}\nANSWER: {answer}\n')

twenty_questions('a cat')

1. QUESTION: Is it a living thing?
ANSWER: yes



UnexpectedModelBehavior: Exceeded maximum retries (1) for result validation